In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import csv
import sys
from datetime import datetime, date
import os

In [ ]:
header = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10.15; rv:96.0) Gecko/20100101 Firefox/96.0'}
FQDN = 'https://www.m******.**/'
responce = requests.get(FQDN)

print("request denied") if responce.status_code != 200 else print("ok")

In [ ]:
def get_html(url, params=None):
    r = requests.get(url, headers=header, params=params)
    return r

In [ ]:
def flatten(L: list) -> list:
    '''рекурентное преобразование списка списков списков... в плоский одноранговый список'''
    if len(L) > sys.getrecursionlimit():
        sys.setrecursionlimit(len(L)*10)
    if L == []:
        return L
    if isinstance(L[0], list):
        return flatten(L[0]) + flatten(L[1:])
    rez = list(L[:1] + flatten(L[1:]))
    return rez

In [ ]:
def str_to_date(str_date: str) -> datetime.date:
    '''принимает дату 17.10.2017 в формате string, возвращает в формате datetime'''
    temp_date = [d for d in str_date.split('.')]
    temp_date.reverse()
    temp_date = '-'.join(temp_date)
    return date.fromisoformat(temp_date)

In [ ]:
def dates_urls_on_page(page_url: str) -> list:
    '''возвращает список дат и ссылок на статьи со страницы'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    dates = [e.text for e in soup.find_all('div', class_='date')]
    short_urls = [e.a.get('href') for e in soup.find_all('p', class_='news_name')]
    fqdn_urls = ['/'.join(base_url.split('/')[:3]) + elem for elem in short_urls]
    
    return dates, fqdn_urls

In [ ]:
def id_urls_on_page(page_url: str) -> list:
    '''возвращает список id и ссылок на Объявления со страницы'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    for e in soup.find_all('div', class_='catalog-section'):
        e.find('div', class_='pagination').extract()
        short_urls = [elem.get('href') for elem in e.find_all('a')]
    fqdn_urls = ['/'.join(base_url.split('/')[:3]) + elem for elem in short_urls[::2]]
    id_list  = [int(i.split('/')[3]) for i in short_urls[::2]]
    
    return id_list, fqdn_urls

In [ ]:
def get_content(page_url: str):
    '''сборщик контента. возвращает 3 списка - текст, внутренние ссылки, ссылки на картинки'''
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')

    text_content = []
    urls_content = []
    img_content = []

    for e in soup.find_all('div', class_='rightContent'):
        # убираем из обработки ненужные теги
        try:
            e.find('div', class_='breadcrumb').extract()
        except (AttributeError ):
            pass
        try:
            e.find('h1').extract()
        except (AttributeError ):
            pass
        try:
            e.find('h3').extract()
        except (AttributeError ):
            pass
        try:
            e.find('div', class_='news_top').extract()
        except (AttributeError ):
            pass
        try:
            e.find('div', class_='pages_block').extract()
        except (AttributeError ):
            pass
        try:
            e.find('div', class_='news-detail-share').extract()
        except (AttributeError ):
            pass
            
        text_content.append(e.text.strip().replace('\n', '').replace('\r', ''))

        for i in e.find_all('img'):
            img_content.append('/'.join(base_url.split('/')[:3]) + i.prettify().split('src=')[1].split('"')[1])

        for i in e.find_all('a'):
            if i.get('href').startswith('/'):
                urls_content.append('/'.join(base_url.split('/')[:3]) + str(i.get('href')))
            elif i.get('href').startswith('http'):
                urls_content.append(i.get('href'))
            else:
                continue

    return text_content, urls_content, img_content

In [ ]:
def flatten_column(df: pd.core.frame.DataFrame, column: str) -> pd.core.frame.DataFrame:
    '''извлекает значения в ячейках датафрейма из списочного вида в текстовый'''

    for idx in range(df.shape[0]):

        if type(df.loc[idx,column]) is list:
            df.loc[idx,column] = ', '.join(df.loc[idx,column]) if len(df.loc[idx,column]) > 0 else ''

    return df

In [ ]:
def read_conf(fqdn: str, start_id: int = 0, start_date: str = '2022-01-01'):
    file_name = 'aggregator.ini'
    domain = '.'.join(fqdn.split('/')[2].split('.')[-2:])
    start_date = date.fromisoformat(start_date)
    if not os.path.exists(file_name):
        open(file_name, 'x')
    with open(file_name) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if domain in row[0]:
                try:
                    start_date = date.fromisoformat(row[1])
                except (ValueError):
                    start_date = start_date
                try:
                    start_id = abs(int(float(row[2])))
                except (ValueError):
                    start_id = start_id
                    
    return start_date, start_id

In [ ]:
def write_conf(fqdn: str, end_id: int, end_date: str = str(datetime.now().date())):
    domain = '.'.join(fqdn.split('/')[2].split('.')[-2:])
    file_name = 'aggregator.ini'
    fin = open(file_name, "rt")
    data = fin.read()
    domain_list = []
    for elem in data.split('\n'):
        domain_list.append(elem.split(',')[0])
        if elem.split(',')[0] == domain:
            data = data.replace(domain + ',' + elem.split(',')[1] +',' + elem.split(',')[2], 
                                domain + ',' + end_date + ',' + str(end_id))
            break
    
    new_row = domain + ',' + end_date + ',' + str(end_id)
    if domain not in domain_list:
        data += new_row if len(domain_list[0]) == 0 else '\n' + new_row

    fin.close()
    fin = open(file_name, "wt")
    fin.write(data)
    fin.close()

In [ ]:
start_date, start_id  = read_conf(FQDN, start_id = 10616) #id объявления 10616 примерно соответствует началу 2022 года

---

###### Раздел "Новости"

In [ ]:
base_url = 'https://www.m*******.**/press-centre/news/?PAGEN_2='
start_page = base_url + str(1)

In [ ]:
# Поиск новых новостей с момента последнего запуска агрегатора
dates, urls = dates_urls_on_page(start_page)

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count)
    dates.extend(dates_urls_on_page(next_page)[0])
    urls.extend(dates_urls_on_page(next_page)[1])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
title=[]
content = []
ext_urls = []
images = []
for page_url in urls[:news_sum]:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    title.append([e for e in soup.find('h1')][0])
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    images.append(get_content(page_url)[2])

In [ ]:
df1 = pd.DataFrame()
df1['Дата'] = dates[:news_sum]
df1['Раздел'] = 'Новости ВУЗА'
df1['Заголовок'] = title
df1['Текст'] = content
df1['Внешние ссылки'] = ext_urls
#df1['Графика'] = images
df1['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df1)[3:5]:
    df1 = flatten_column(df1,column)
df1.head(3)

---

###### Раздел "Новости К"

In [ ]:
base_url = 'https://www.m******.**/press-centre/news/faculty/?PAGEN_2='
start_page = base_url + str(1)

In [ ]:
# Поиск новых новостей Корпусов с момента последнего запуска агрегатора
dates, urls = dates_urls_on_page(start_page)

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count)
    dates.extend(dates_urls_on_page(next_page)[0])
    urls.extend(dates_urls_on_page(next_page)[1])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
title=[]
content = []
ext_urls = []
images = []
for page_url in urls[:news_sum]:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    title.append([e for e in soup.find('h3')][0])
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    images.append(get_content(page_url)[2])

In [ ]:
df2 = pd.DataFrame()
df2['Дата'] = dates[:news_sum]
df2['Раздел'] = 'Новости Корпусов'
df2['Заголовок'] = title
df2['Текст'] = content
df2['Внешние ссылки'] = ext_urls
#df2['Графика'] = images
df2['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df2)[3:5]:
    df2 = flatten_column(df2,column)
df2.head(3)

---

###### Раздел "Новости Ф"

In [ ]:
base_url = 'https://www.m******.**/press-centre/news/branch/?PAGEN_2='
start_page = base_url + str(1)

In [ ]:
# Поиск новых новостей Корпусов с момента последнего запуска агрегатора
dates, urls = dates_urls_on_page(start_page)

count=1
while str_to_date(dates[-1]) > start_date:
    count += 1
    next_page = base_url + str(count)
    dates.extend(dates_urls_on_page(next_page)[0])
    urls.extend(dates_urls_on_page(next_page)[1])

# Количество новых статей с даты последнего запуска агрегатора
news_sum = sum([True for date_news in dates if str_to_date(date_news) > start_date])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} статей.')

In [ ]:
title=[]
content = []
ext_urls = []
images = []
for page_url in urls[:news_sum]:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    title.append([e for e in soup.find('h3')][0])
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    images.append(get_content(page_url)[2])

In [ ]:
df3 = pd.DataFrame()
df3['Дата'] = dates[:news_sum]
df3['Раздел'] = 'Новости Филиалов'
df3['Заголовок'] = title
df3['Текст'] = content
df3['Внешние ссылки'] = ext_urls
#df3['Графика'] = images
df3['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df3)[3:5]:
    df3 = flatten_column(df3,column)
df3.head(3)

---

###### Раздел "Объявления"

In [ ]:
base_url = 'https://www.m*******.**/press-centre/ads/?PAGEN_2='
start_page = base_url + str(1)

In [ ]:
# Поиск новых новостей с момента последнего запуска агрегатора
ids, urls = id_urls_on_page(start_page)

count=1
while ids[-1] > start_id:
    count += 1
    next_page = base_url + str(count)
    ids.extend(id_urls_on_page(next_page)[0])
    urls.extend(id_urls_on_page(next_page)[1])

# Количество новых объявлений с момента последнего запуска агрегатора
news_sum = sum([True for i in ids if i > start_id])

print(f'С даты последнего запуска агрегатора [{start_date}] опубликовано {news_sum} Объявлений.')

In [ ]:
title=[]
content = []
ext_urls = []
images = []
for page_url in urls[:news_sum]:
    response = get_html(page_url)
    soup = BeautifulSoup(response.text, 'lxml')
    title.append([e for e in soup.find('h1')][0])
    content.append(get_content(page_url)[0])
    ext_urls.append(get_content(page_url)[1])
    images.append(get_content(page_url)[2])

In [ ]:
df4 = pd.DataFrame()
df4['Номер объявления'] = ids[:news_sum]
df4['Раздел'] = 'Объявления'
df4['Заголовок'] = title
df4['Текст'] = content
df4['Внешние ссылки'] = ext_urls
#df4['Графика'] = images
df4['URL статьи'] = urls[:news_sum]

In [ ]:
# корректируем столбцы 'Текст' и 'Внешние ссылки'
for column in list(df4)[3:5]:
    df4 = flatten_column(df4,column)
df4.head(3)

In [ ]:
file_name = base_url.split('/')[2].split('.')[1] + '_' + str(datetime.now().date()) + '.xlsx'
with pd.ExcelWriter(file_name) as writer:
    df1.to_excel(writer, sheet_name='Новости ВУЗа', index=False)
    df2.to_excel(writer, sheet_name='Новости Корпусов', index=False)
    df3.to_excel(writer, sheet_name='Новости Филиалов', index=False)
    df4.to_excel(writer, sheet_name='Объявления', index=False)

In [ ]:
#обновление хранимого id-объявлений и даты в файле aggregator.ini
id_ads = ids[:news_sum][0] if news_sum > 0 else ids[0]

write_conf(FQDN, id_ads)

---